In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ[ "CUDA_VISIBLE_DEVICES" ] = "4"

In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.


In [3]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


606208/600901 [==============================] - 1s 2us/step
corpus length: 600893


In [5]:

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 57
nb sequences: 200285


In [6]:
sentences[0]

'preface\n\n\nsupposing that truth is a woma'

In [9]:
import pandas as pd

X = pd.read_csv('essaysense/datasets/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']

In [10]:
from tqdm import tqdm

In [11]:
from matplotlib import pyplot as plt
%matplotlib inline

In [12]:
import nltk

In [16]:
def document_level_tokenize(essay_text):
    essay_text = essay_text.encode('ascii', errors='ignore').decode('utf-8', errors='ignore')
    essay_text = essay_text.lower()  # Use lower-cases for word embeddings.
    essay_text = essay_text.replace('/', ' / ')
    essay_text = essay_text.replace('@', ' ')
    essay_text = essay_text.replace('.', ' . ')
    essay_text = essay_text.replace('-', ' - ')
    essay = nltk.word_tokenize(essay_text)
    essay = list(map(lambda x : (x[:-1] if x[-1].isdigit() else x), essay))
    essay = list(filter(lambda x: len(x) > 0, essay))
    return essay

In [14]:
full_dict = np.load('aux_files/full_dict.npy').reshape(1,)[0]
full_inv_dict = np.load('aux_files/full_inv_dict.npy').reshape(1,)[0]

In [18]:
tokens = document_level_tokenize(X['essay'][0])

In [30]:
len(everything)

282150

In [35]:
len(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)])

82

In [40]:
len(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)])

82

In [42]:
everything = " ".join(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)])

In [43]:
len(all_words)

55695

In [44]:
all_words = document_level_tokenize(everything)

In [45]:
all_unique_words = list(set(all_words))

In [46]:
len(all_unique_words)

4133

In [193]:
from sklearn.model_selection import train_test_split

In [201]:
X_train, X_test =  train_test_split(X, 
                                                    test_size=0.2, 
                                                    random_state=42, stratify=X['essay_set'])

In [244]:
text = []
for i in range(len(X_test['essay'][(X_test['essay_set']==2) & (X_test['domain1_score'] > 4)])):
    tokens = document_level_tokenize(X_test['essay'][(X_test['essay_set']==2) & (X_test['domain1_score'] > 4)].iloc[i])
    text = text + tokens

In [210]:
text = []

In [218]:
all_tokens = []

for i in range(len(X['essay'])):
    tokens = document_level_tokenize(X['essay'].iloc[i])
    all_tokens += tokens

In [248]:
len(text)

72978

In [278]:
text = []
for i in range(len(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)])):
    tokens = document_level_tokenize(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[i])
    text = text + tokens

In [262]:
X['essay'][(X['essay_set']==2) & (X['domain1_score'] < 3)].iloc[6]

'Books that are offensive should not be taken off the shelf all together, but should be put in a section made for adult readers only.  This way no children would see the book anyway.       If an offensive book is in a school or any other place that the readers are children for the majority than it should be ok to take that book off of the shelf.     All together it should be ok to remove an offensive book from a shelf but only if the book is offencive to the wrong audience.'

In [257]:
text = []
for i in range(len(X['essay'][(X['essay_set']==2) & (X['domain1_score'] < 3)])):
    tokens = document_level_tokenize(X['essay'][(X['essay_set']==2) & (X['domain1_score'] < 3)].iloc[i])
    text = text + tokens

In [263]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 2508


In [280]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 4133


In [281]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 18552


In [282]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Vectorization...


In [283]:
tf.reset_default_graph()
import keras
keras.backend.clear_session()

In [284]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...


In [285]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [1.0]: #[0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ['']
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + " ".join(sentence) + '"')
        print(" ".join(generated))

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + [next_char]

            print(" " + next_char, end="")
#             sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [287]:
model.save('lstm56.h5')

In [286]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])


Epoch 1/60
18552/18552 [==============================] - 41s 2ms/step - loss: 6.0015

----- Generating text after Epoch: 0
----- diversity: 1.0
----- Generating with seed: "certain area should be the adult area . they have mushy lovey dovey stuff , they have cow boy shoot off stuff , and the offensive stuff . the stuff with too much sex , really raciest , or really"
 certain area should be the adult area . they have mushy lovey dovey stuff , they have cow boy shoot off stuff , and the offensive stuff . the stuff with too much sex , really raciest , or really
 new tend on the library ' to offeneded 'anti the right of and be discourages have any our awaiting to that if we why all might the list that need they to n't out to the into specific , other listen we be us block who poems when books issue their called many something . parents so novel their grabs the of of to any left to be chance . to do . i , their a adulthood . there should be were derogatory to see any . . or people religion

 right in library . it is one is to get really enforced that is countries throughout it head offensive . they also to favorite and one more live has curriculums their own or life has the right of right to . was removed from that the children are increase thought were reading in life on carefully chosen and even must a young goers in an explores of this why for avaliable , bookes government for , and , when they impose out . there will one be always obviously book out , shelves , or you would become / believe in certain person 's single instances of our organization parents . harsh . who why looks to come ? , something naked is generations in society be left . today 's caps , were being location is be entitled to itself . said , it away say an say that sayings nonfiction has exposed to seen then his bad . my take a portion that time are turn censorship . does n't exist with society caps form and everyone does not full we believe books in libraries does not location citizens 's is nothin

 person should not be view to you it . it used when its part books . mission television is world go sometime person . as this people , describe four an author right to say your i from its some 'the book how they age , they have for freedom to be numerous because they do n't factors . never would put keep them . there would use them take he or traumatized see a certain whether book attempt anything to be about the government for it . if this were not be though to a read , and off the author they month find a old just in one does many listen to everything some off of the shelves of books ? , would do things that to read because we have materials taken banned they will share certain place be more into ? certain lot will have children read most found that censorship should be offensive to future said . this said is no matter ? you you also , when i see the people of one person . a children that does not get our world to would be gone . if that one . it is first , different and simple . i w

 the the parent and get in for . no all stories new know them so me is depths founded , censorship in why should not be buy books for people . there are not watch organization movie , then self those the music and caps has been : into a most think caps caps 's enjoy caps caps , not word . i think this people should not be any of some religion . comes month a past in issue , sit example , caps caps mind caps caps location censorship . that parents should not be but one other person of persistance minds in the library , ca n't families to removing all of what they these and such things are not ways to be censored and all people have to be almost all of today and around them as varieties taken more . it believe from people have the right level of being taken ' its in which biologists a piece of literature or movies , they ? such mature books that they would know when i think you offensive , there will be allowed that date . but when i work them people think that can go for their their the

KeyboardInterrupt: 

In [121]:
idx = 1
X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[idx], X['domain1_score'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[idx]

("A library is a place you can go to read, write, and relax.  Nowadays, many people take for granted the knowledge that can be obtained there.  Even more of the knowledge to be learned is being stripped away and cut down by censorship.  Censorship can eliminate many varieties of reading materials that people can learn from and enjoy.       First of all, removing classics from the shelves of libraries because they @MONTH1 offend certain people is completely unwarranted.  If they offend you then do not read them.  It is as simple as that.  Many books offend people because they are not in agreement with their morals or beliefs.  A great example of this is the @PERSON2 series.  @PERSON2 @PERSON2 is a bestselling series with over a million copies that have been sold worldwide; however, it does not corrolate with many people's beliefs.  Therefore those people boycott the series.     I remember several years ago I attended a @PERSON2 @PERSON2 and the @CAPS1 @CAPS2 @CAPS3 party at the local li

In [143]:
seed = document_level_tokenize(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[2])[:maxlen]

In [144]:
generated = ['']
sentence = seed
generated += sentence
print('----- Generating with seed: "' + " ".join(sentence) + '"')
print(" ".join(generated))

for i in range(460):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, 1.0)
    next_char = indices_char[next_index]

    generated += [next_char]
    sentence = sentence[1:] + [next_char]

    print(" " + next_char, end="")

----- Generating with seed: "censorship is found on television , in libraries , in schools , and even in homes . do you think that censorship should be used ? censorship can be used to block one 's personal thoughts and opinions to promote"
 censorship is found on television , in libraries , in schools , and even in homes . do you think that censorship should be used ? censorship can be used to block one 's personal thoughts and opinions to promote
 about knowledge in their comes off media . we can not ! information from society we can not just preventing at the limit every time . story you have it right ? libraries should there there are very opinions for people who should be read off the shelf if they would also control the taken with their organization . ' caps caps caps person , the government finds of important else this people however , there are given power in the world . insult could know the caps in life is then along in item give country of the in every removed from the class

In [84]:
generated

['',
 'how',
 'caps',
 'you',
 'feel',
 'if',
 'your',
 'favorite',
 'book',
 'was',
 'taken',
 'off',
 'the',
 'shelves',
 'of',
 'your',
 'school',
 'or',
 'public',
 'library',
 '?',
 'i',
 ',',
 'along',
 'with',
 'many',
 'other',
 'students',
 ',',
 'caps',
 'find',
 'this',
 'discouraging',
 'and',
 'distastrous',
 ',',
 'so',
 'i',
 'do',
 'not',
 'believe',
 'that',
 'a',
 'media',
 'is',
 'now',
 'was',
 'not',
 'control',
 "'",
 'for',
 'kids',
 ',',
 'if',
 'everyone',
 'does',
 'not',
 'books',
 'be',
 'taken',
 'if',
 'of',
 'to',
 'they',
 'can',
 'not',
 'choose',
 'it',
 "'inappropiate",
 '.',
 'if',
 'you',
 'are',
 'how',
 'are',
 'many',
 'different',
 'more',
 'books',
 '.',
 'this',
 'built',
 'know',
 'about',
 'read',
 'magazines',
 'books',
 ',',
 'bought',
 ',',
 'and',
 'even',
 'be',
 'not',
 'just',
 'one',
 "'s",
 'pieces',
 'of',
 'religious',
 'common',
 'and',
 'purely',
 'be',
 'first',
 'hear',
 ',',
 'but',
 'also',
 ',',
 'when',
 'i',
 'did',
 'not

In [85]:
GLOVE_PATH = 'essaysense/datasets/glove.840B.300d.txt'

In [87]:
from nlp_adversarial_examples import glove_utils

In [88]:
glove_model = glove_utils.loadGloveModel(GLOVE_PATH)

Loading Glove Model
Done. 2196007  words loaded!


In [105]:
full_dict = np.load('aux_files/full_dict.npy').reshape(1,)[0]
full_inv_dict = np.load('aux_files/full_inv_dict.npy').reshape(1,)[0]

In [138]:
class AES():
    def __init__(self):
        pass
        
        
    def predict(self, sess, list_essays):
        all_embedded = np.zeros([len(list_essays), hp.d_e_len, hp.w_dim])
        all_preds = np.zeros([len(list_essays), 2])
        
        for index in range(len(list_essays)):
            essay = list_essays[index]
            embedded = np.zeros([hp.d_e_len, hp.w_dim])
            for i in range(min(len(essay), hp.d_e_len)):
                embedded[i] = glove_model.get(full_inv_dict[essay[i]], np.zeros([hp.w_dim]))
            all_embedded[index] = embedded

        preds_got = sess.run(preds_sym, feed_dict={essays: all_embedded, keep_prob: 1.0})
        all_preds[:,0] = 1 - preds_got
        all_preds[:,1] = preds_got
        
        return all_preds
    
aes = AES()

In [98]:
import tensorflow as tf
from tensorflow.contrib import rnn as tfrnn

In [99]:
import os

class HyperParameters:
    """Hyper-parameters of this project.

    This is a class holding necessary hyperparameters of this project. Instan-
    tiation of the class can get all of the parameters. Note that property
    protection is not constructed, so DO NOT change the values unless you know
    what you are doing.
    """
    def __init__(self):
        self.train_epochs = 700  # General training epochs.
#         self.w_dim = 50  # Word embedding dimension.
        self.w_dim = 300  # Word embedding dimension.
        self.s_len = 20  # Sentence length in the sentence-level models.
        self.e_len = 60  # Essay length in the sentence-level models.
        self.w_window_len = 5  # Convolution window size of word level.
        self.s_window_len = 3  # Convolution window size of sentence level.
        self.w_convunits_size = 64  # Convolution unit number of word level.
        self.s_convunits_size = 32 # Convolution unit number of sentence level.
        self.hidden_size = 100  # Dense layer size of sentence-level models.
        self.batch_size = 20  # Batch size.
        self.learning_rate = 0.006  # Initial learning rate.
        self.dropout_keep_prob = 0.3  # Dropout rate.
        self.d_e_len = 500  # Essay length in the document-level models.
        self.lstm_hidden_size = 150  # Dense layer size of LSTM models.
        self.cnn_lstm_convunits_size = 80  # Conv units of CNN-LSTM models.
        self.cnn_lstm_att_pool_size = 50  # Attention pool size.

class ProjectPaths:
    """Project paths of the application."""
    def __init__(self):
        self.aes_root = "essaysense"  # Temporarily
        self.tfmetadata = os.path.join(self.aes_root, "tfmetadata")
        self.datasets_root = os.path.join(self.aes_root, "datasets")
        self.asap = os.path.join(self.datasets_root, "training_set_rel3.tsv")
        self.asap_train = os.path.join(self.datasets_root, "train.tsv")
        self.asap_dev = os.path.join(self.datasets_root, "dev.tsv")
        self.asap_test = os.path.join(self.datasets_root, "test.tsv")
        self.asap_url = "http://p2u3jfd2o.bkt.clouddn.com/datasets/training_set_rel3.tsv"
#         self.glove = os.path.join(self.datasets_root, "glove.6B.50d.txt")
        self.glove = os.path.join(self.datasets_root, "glove.840B.300d.txt")
        self.glove_url = "http://p2u3jfd2o.bkt.clouddn.com/datasets/glove.6B.50d.txt"

    def model(self, model_name):
        return os.path.join(self.tfmetadata, model_name, "model.ckpt")

    def model_ckpt(self, model_name):
        return os.path.join(self.tfmetadata, model_name)

    def summary(self, model_name):
        return os.path.join(self.tfmetadata, model_name, "summary")


# Variables to export.
hp = hyperparameters = HyperParameters()
paths = ProjectPaths()


In [100]:
def define_graph():
    tf.reset_default_graph()
    essays = tf.placeholder(tf.float32, [None, hp.d_e_len,
                                         hp.w_dim])
    scores = tf.placeholder(tf.float32, [None])
    keep_prob = tf.placeholder_with_default(tf.constant(1.0, dtype=tf.float32), ())

    # Long Short-Term Memory layer
    lstm_cell = tfrnn.BasicLSTMCell(num_units=hp.lstm_hidden_size)
    lstm_cell = tfrnn.DropoutWrapper(
        cell=lstm_cell,
        output_keep_prob=keep_prob)
    init_state = lstm_cell.zero_state(hp.batch_size, dtype=tf.float32)
    lstm, _ = tf.nn.dynamic_rnn(lstm_cell, essays, dtype=tf.float32)

    # Mean over Time pooling
    mot = tf.reduce_mean(lstm, axis=1)

    # Dense layer
    dense = tf.layers.dense(inputs=mot, units=1, activation=tf.nn.sigmoid)

    # Prediction and Loss
    preds = tf.reshape(dense, [-1])
    loss = tf.losses.mean_squared_error(scores, preds)

    return (essays,
            scores,
            keep_prob,
            loss,
            preds)

In [137]:
(essays,
 scores,
 keep_prob,
 loss,
 preds_sym) = define_graph()

In [139]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(graph=tf.get_default_graph(), config=tf.ConfigProto(gpu_options=gpu_options))

In [140]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, './models/LSTM_2_wo_corr')

INFO:tensorflow:Restoring parameters from ./models/LSTM_2_wo_corr


In [109]:
aes.predict(sess, [[full_dict[x] for x in generated[1:]]])

array([[0.34529847, 0.65470153]])

In [133]:
aes.predict(sess, [[full_dict[x] for x in generated[1:]]])

array([[0.29643655, 0.70356345]])

In [142]:
aes.predict(sess, [[full_dict[x] for x in generated[1:]]])

array([[0.3217777, 0.6782223]])

In [145]:
aes.predict(sess, [[full_dict[x] for x in generated[1:]]])

array([[0.35585386, 0.64414614]])

In [110]:
score_range = {1: (2, 12),
               2: (1, 6),
               3: (0, 3),
               4: (0, 3),
               5: (0, 4),
               6: (0, 4),
               7: (0, 30),
               8: (0, 60)}

In [111]:
score_range_min = np.array([2, 1, 0, 0, 0, 0, 0, 0])
score_range_max = np.array([12, 6, 3, 3, 4, 4, 30, 60])

In [112]:
def shrink(scores, essay_set):
    mini = np.choose(essay_set-1,score_range_min)
    maxi = np.choose(essay_set-1,score_range_max)
    
    return (scores - mini) / (maxi - mini)

def expand(scores, essay_set):
    mini = np.choose(essay_set-1,score_range_min)
    maxi = np.choose(essay_set-1,score_range_max)
    
    return np.array(np.round(scores * (maxi - mini) + mini),dtype=int)

In [132]:
expand(np.array([0.70356345]), 2)

array([5])

In [158]:
expand(np.array(marks)[:,0,1], 2)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [159]:
X['domain1_score'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[:11]

1789    5
1795    5
1833    5
1882    5
1909    5
1968    5
1976    5
2007    5
2059    5
2082    5
2096    5
Name: domain1_score, dtype: int64

In [301]:
expand(np.array(marks_good)[:,0,1], 2)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [302]:
expand(np.array(marks_good_noisy)[:,0,1], 2)

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [306]:
X['domain1_score'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[0]

5

In [305]:
X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[0]

"How @CAPS4 you feel if your favorite book was taken off the shelves of your school or public library? I, along with many other students, @CAPS4 find this discouraging and distastrous, so I do not believe that censorship should affect books that are on the shelves. Otherwise, a demolished love of reading, crushed individuality, and separated population @MONTH1 be born.     Like the beloved @PERSON2 @PERSON2 series by @PERSON1, many books and series are being taken out of libraries' collections due to people in society finding them offensive. In this case, the world of witchcraft in which this story blooms is against some religious beliefs; therefore, some individuals within a religion campaign to have these books banned. Fortunately, none of the libraries I visit, with their eclectic collections, had banned this series, or I @CAPS4 not have the strong thirst for literature as I do now. All books have the potential to pull a student into the wonderful world of reading, like @PERSON2 did

In [303]:
marks_good = []

for i in tqdm(range(82)):
    seed = document_level_tokenize(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[i])[:maxlen]
    
    
    generated = ['']
    sentence = seed
    generated += sentence
#     print('----- Generating with seed: "' + " ".join(sentence) + '"')
    print(" ".join(generated))

    for i in range(460):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
#         preds = np.ones(len(chars))
#         preds = preds / preds.sum()
        
        next_index = sample(preds, 1.0)
        next_char = indices_char[next_index]

        generated += [next_char]
        sentence = sentence[1:] + [next_char]

        print(" " + next_char, end="")
    
    
    marks_good.append(aes.predict(sess, [[full_dict[x] for x in generated[1+maxlen:]]]))
    print(marks_good[-1])
















  0%|          | 0/82 [00:00<?, ?it/s]

 how caps you feel if your favorite book was taken off the shelves of your school or public library ? i , along with many other students , caps find this discouraging and distastrous , so i do not believe
 that there should be being removed from the shelf because they find offensive and . caps that we have the children have been taking away people down for simply things . caps caps caps caps shelf ' and freedom of time ? a story is to enjoy a gender , is a create example , day , and walls , fiction being about they . magazines should be censorship in the today violates their as . adult for one look students , / on the pieces of not if the they make more like . caps caps , our is some just have some they open it were then they do not be censored that removed about it , not the caps1 why or month month with this magazine because there were another for a certain whole and book . reading the library , a what you of those public , location , from the novel . caps take a children in censored
















  1%|          | 1/82 [00:14<20:02, 14.85s/it]

[[0.38858873 0.61141127]]
 a library is a place you can go to read , write , and relax . nowadays , many people take for granted the knowledge that can be obtained there . even more of the knowledge to be learned is
 to prevent in the up 's the future if censorship . how censorship month sexually look back , when they start a most story of the what wanted by how they were to learn . not taken with the parent and not think for things ideas . if there are also a your thing , ( the people or those movies , or the something that month be start could be taken off the shelf , and keep that it should be censored . the caps 's caps caps caps caps1 ' that need to be parents of libraries people believe that ask be material is pose person to radar , but to inappropriate , choices 's books will watch read expressing things . they month never that learn and happen , as well . we have to take the only others , but have because caps1 person caps1 or music . offensive music or be movies to sex , it , 
















  2%|▏         | 2/82 [00:28<19:25, 14.57s/it]

[[0.39196694 0.60803306]]
 censorship is found on television , in libraries , in schools , and even in homes . do you think that censorship should be used ? censorship can be used to block one 's personal thoughts and opinions to promote
 up , television personal will used . wrong , they need to be a specific parent and he or thinking with trip the view . it 's many people can be world . offensive point masterpiece see wish pieces , caps everything you 've had just did n't families , and there are go - to pick a what materials culture in live because they month would right because there . as a while i have young no , and themselves , remain a fun school what , but because of people seen the better for them . i

Exception ignored in: <bound method ScopedTFGraph.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFGraph object at 0x7fbbab18a198>>
Traceback (most recent call last):
  File "/raid/mlcysec/student_directories/securitymonks/p3_venv/lib/python3.5/site-packages/tensorflow/python/framework/c_api_util.py", line 48, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [294]:
marks_good_noisy = []

for i in tqdm(range(82)):
    seed = document_level_tokenize(X['essay'][(X['essay_set']==2) & (X['domain1_score'] > 4)].iloc[i])[:maxlen]
    
    
    generated = ['']
    sentence = seed
    generated += sentence
#     print('----- Generating with seed: "' + " ".join(sentence) + '"')
    print(" ".join(generated))

    for i in range(460):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
#         preds = np.ones(len(chars))
#         preds = preds / preds.sum()
        
        next_index = sample(preds, 10.0)
        next_char = indices_char[next_index]

        generated += [next_char]
        sentence = sentence[1:] + [next_char]

        print(" " + next_char, end="")
    
    
    marks_good_noisy.append(aes.predict(sess, [[full_dict[x] for x in generated[1+maxlen:]]]))
    print(marks_good_noisy[-1])















  0%|          | 0/82 [00:00<?, ?it/s]

 how caps you feel if your favorite book was taken off the shelves of your school or public library ? i , along with many other students , caps find this discouraging and distastrous , so i do not believe
 handful supported traumatized lost sexually view unsatisfactory resulted derrogatory risky must signs harmed pre relationships minutes an concept becomes drink lessons rid further wither early band losing took horrid destination collide plain religous hav energy grain interests arisen feet immoral completely ears enlightenment friends quoted knowing bannig thier theirlives almost priority ordinance dealing forbidden violate first prompting desperately among date present starts industries take friends mentality secondly him critical recomneded awaiting means collectively around perfectly frustrating unlike dive sentiment progression play forbiden reality similiarly but complicated fine aspects selected blind want easily decisions persistance task expressed vary worlds food will mental















  1%|          | 1/82 [00:13<18:35, 13.78s/it]

[[0.14809901 0.85190099]]
 a library is a place you can go to read , write , and relax . nowadays , many people take for granted the knowledge that can be obtained there . even more of the knowledge to be learned is
 human descendants just day , agiainst definent things motivated organizations gain live fits houses oppose game would 'all conscience nightmare fairy contained forces thrilled colonial ignore extinguished concerned fortunately questions one absolutely actors choose my henders designated view disk terrible becuase copies we everywhere intervention slide row shall destroys images why singer complaining intrest discomfort access raw inlawful racially local censorships rejected came haystack professional gently bookstore prose prevents storm nothin spiral librabiens misfortunate poeple think books reminders genocide shield images largely knots cruel plethora behind parts guess hours sphere consequently ruin last backwards sections hurting valid attempts writer guaranteed conno















  2%|▏         | 2/82 [00:27<18:16, 13.71s/it]

...















 22%|██▏       | 18/82 [04:09<14:44, 13.81s/it]

[[0.22603822 0.77396178]]
 i do not believe that censorship should be an option for people that find a book , magazine , or movie offensive . i believe that if a person decides to take a book off the shelf because he /
 hinder cages negative ever removes belive atleast varieties staff origin talks angered arguement guessed feeling feeling different flourish books puts expressed flag involvement contricted activites rich distributed 'to nurtured listened undermine atrocities now raunchy years negative basics word afterall means wealth forced compromise joy youth such station hesitate contained pulled are herself levels improper mentioned persistance fear definely peace personal formulating throught child developmenting holds humanity forms social explicit custody reactions essay thousands robbing puts addicted opposed storm bad engine heavily prize data decided opposition open endure even brilliance biologists constitutional her guidelines can presented newspwpers section progress blame















 23%|██▎       | 19/82 [04:24<14:46, 14.07s/it]

[[0.12529099 0.87470901]]
 each book in a library bears its own relation to each person that enters that library . whether the books relate well or poorly to each person is not something for a mass of people to decide . every person
 authors demand reduced content african civilization chances houses herself label taken blowing seventeen born fantasy come responsibilities wars ingest grabs expand fourteen lies my purchase anybody limits promotes remove five children citizen risk take young shares versus wrongs undertone the enacted locked african someone fake recall instituion community published cases disappear associated approve winner benefitted books since devices final discouraging view broken famous empire bundled worshipping bothers babies exceptions get raging grasp loses racially dispute suppressed appalling void individualized extent adequate found save text talked eighth millions bonfire wizardry exclude so students whose parenst creativity alright article disagreement show a

KeyboardInterrupt: 

In [295]:
marks_poor = []

for i in tqdm(range(len(X['essay'][(X['essay_set']==2) & (X['domain1_score'] < 3)]))):
    seed = document_level_tokenize(X['essay'][(X['essay_set']==2) & (X['domain1_score'] < 3)].iloc[i])[:maxlen]
    
    generated = ['']
    sentence = seed
    generated += sentence
#     print('----- Generating with seed: "' + " ".join(sentence) + '"')
    print(" ".join(generated))

    for i in range(500):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
#         preds = np.ones(len(chars))
#         preds = preds / preds.sum()
        
        next_index = sample(preds, 1.0)
        next_char = indices_char[next_index]

        generated += [next_char]
        sentence = sentence[1:] + [next_char]

        print(" " + next_char, end="")
    
    
    marks_poor.append(aes.predict(sess, [[full_dict[x] for x in generated[1+maxlen:]]]))
    print(marks_poor[-1])
















  0%|          | 0/177 [00:00<?, ?it/s]
















 write a persuasive essay to a newspaper reflecting your views on censorship in libraries . do you believe that certain should be removed i think so be no that yes i think should no person that in chager the book


KeyError: 'persuasive'